# Rocket Problem with AeroSandbox Dynamics Engine

## Overview of the Dynamics Engine

### Overview & Motivation

In aerospace problems, we often end up with system dynamics that look (relatively) similar: our objects of interest are (roughly) rigid bodies moving around in 3D space (which we sometimes approximate as 2D or 1D).

While there's nothing wrong with implementing the dynamics manually for a problem [as we did in the previous tutorial](./01%20-%20Rocket%20Problem%20with%20Manual%20Dynamics.ipynb), it's a bit tedious and error-prone to manually implement the same dynamics every time we want to simulate an aerospace system.

So, AeroSandbox provides some (optional) shorthands that can be used to save hundreds of lines of code.

### Definitions, Coordinate Systems, and Assumptions

AeroSandbox's 3D dynamics engine essentially implements the full (nonlinear) equations of motion from Chapter 9 of [*"Flight Vehicle Aerodynamics"* by Mark Drela](https://mitpress.mit.edu/books/flight-vehicle-aerodynamics), along with the same (standard) coordinate system assumptions.

First, we define relevant axes systems:

* Earth Axes: Using the North, East, Down (NED) convention. Note that this implies some things:
    * $z_e$ **points down!** If you want altitude, you want $-z_e$.
    * Assumes negligible local curvature of the earth. Don't use this for your hypersonic vehicles without corrections.
* Body Axes: An axis system that is "painted on" (i.e., rotates with) the vehicle. Uses the convention where:
    * $x_b$ points forward on the vehicle.
    * $y_b$ points rightward on the vehicle.
    * $z_b$ points down on the vehicle.

  In other words, body axes are equivalent to geometry axes rotated 180° about the $y_g$ axis.

Specifically, we parameterize the state of a rigid 3D body in space with the following 12 state variables:

* $x_e$: $x$-position, in Earth axes. [meters]
* $y_e$: $y$-position, in Earth axes. [meters]
* $z_e$: $z$-position, in Earth axes. [meters]
* $u$: $x$-velocity, in body axes. [m/s]
* $v$: $y$-velocity, in body axes. [m/s]
* $w$: $z$-velocity, in body axes. [m/s]
* $\phi$: roll angle. Uses yaw-pitch-roll Euler angle convention. [rad]
* $\theta$: pitch angle. Uses yaw-pitch-roll Euler angle convention. [rad]
* $\psi$: yaw angle. Uses yaw-pitch-roll Euler angle convention. [rad]
* $p$: $x$-angular-velocity, in body axes. [rad/sec]
* $q$: $y$-angular-velocity, in body axes. [rad/sec]
* $r$: $z$-angular-velocity, in body axes. [rad/sec]

Force inputs to the system can be declared using the following inputs (in body axes):

* $X$: $x_b$-direction force. [N]
* $Y$: $y_b$-direction force. [N]
* $Z$: $z_b$-direction force. [N]
* $L$: Moment about the $x_b$-axis. Assumed these moments are applied about the center of mass. [Nm]
* $M$: Moment about the $y_b$-axis. Assumed these moments are applied about the center of mass. [Nm]
* $N$: Moment about the $z_b$-axis. Assumed these moments are applied about the center of mass. [Nm]

Mass properties are also defined:
* $m$: Mass of the body. [kg]
* $I_{xx}$: Respective component of the (symmetric) moment of inertia tensor.
* $I_{yy}$: Respective component of the (symmetric) moment of inertia tensor.
* $I_{zz}$: Respective component of the (symmetric) moment of inertia tensor.
* $I_{xy}$: Respective component of the (symmetric) moment of inertia tensor.
* $I_{xz}$: Respective component of the (symmetric) moment of inertia tensor.
* $I_{yz}$: Respective component of the (symmetric) moment of inertia tensor.

As are a few other quantities:
* $g$: Magnitude of gravitational acceleration. Assumed to act in the $z_e$ ("downward") direction, so a nominal value would be `9.81`. [m/s^2]
* $h_x$: $x_b$-component of onboard angular momentum (e.g. propellers), in body axes. [kg*m^2/sec]
* $h_y$: $y_b$-component of onboard angular momentum (e.g. propellers), in body axes. [kg*m^2/sec]
* $h_z$: $z_b$-component of onboard angular momentum (e.g. propellers), in body axes. [kg*m^2/sec]

## Example on Rocket Problem

Here, we pose the same rocket optimal control problem using the AeroSandbox dynamics engine.

In [2]:
import aerosandbox as asb
import aerosandbox.numpy as np

### Environment
opti = asb.Opti()

### Time discretization
N = 500  # Number of discretization points
time_final = 100  # seconds
time = np.linspace(0, time_final, N)

### Constants
mass_initial = 500e3  # Initial mass, 500 metric tons
z_e_final = -100e3  # Final altitude, 100 km
g = 9.81  # Gravity, m/s^2
alpha = 1 / (300 * g)  # kg/(N*s), Inverse of specific impulse, basically - don't worry about this

dyn = asb.DynamicsPointMass1DVertical(
    mass_props=asb.MassProperties(mass=opti.variable(init_guess=mass_initial, n_vars=N)),
    z_e=opti.variable(init_guess=np.linspace(0, z_e_final, N)),  # Altitude (negative due to Earth-axes convention)
    w_e=opti.variable(init_guess=-z_e_final / time_final, n_vars=N),  # Velocity
)

dyn.add_gravity_force(g=g)
thrust = opti.variable(init_guess=g * mass_initial, n_vars=N)
dyn.add_force(Fz=-thrust)

dyn.constrain_derivatives(
    opti=opti,
    time=time,
)

### Fuel burn
opti.constrain_derivative(
    derivative=-alpha * thrust,
    variable=dyn.mass_props.mass,
    with_respect_to=time,
    method="midpoint",
)

### Boundary conditions
opti.subject_to([
    dyn.z_e[0] == 0,
    dyn.w_e[0] == 0,
    dyn.mass_props.mass[0] == mass_initial,
    dyn.z_e[-1] == z_e_final,
])

### Path constraints
opti.subject_to([
    dyn.mass_props.mass >= 0,
    thrust >= 0
])

### Objective
opti.minimize(-dyn.mass_props.mass[-1])  # Maximize the final mass == minimize fuel expenditure

### Solve
sol = opti.solve(verbose=False)
print(f"Solved in {sol.stats()['iter_count']} iterations.")
dyn.substitute_solution(sol)

print(dyn)

Solved in 16 iterations.
DynamicsPointMass1DVertical instance:
	State variables:
		     z_e: [ 0.00000000e+00 -2.73700089e+01 -1.2...
		     w_e: [ 3.49175374e-248 -2.73152689e+002 -7...
	Control variables:
		    Fz_e: [-1.16379952e+08 -1.13926397e+09 -8.2...


Cool, so the problem solves!

This is an instance where the abstraction provided by the AeroSandbox dynamics engine isn't really needed. The dynamics here are so simple (they're 1-dimensional!) that we can just as easily implement our own integrators.

But where the dynamics engine really shines is when problems get more complicated - 2D and 3D problems with gyroscopic effects, interacting flight dynamics modes, etc. Let's take a look at an example in the following tutorial.